<a href="https://colab.research.google.com/github/Shreyojit/AI-Finance-Agent-/blob/main/AI_Finance_Agent_(LlamaIndex_%7C_LlaMa_3_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI Finance Agent (LlamaIndex | LlaMa-3.1)
Table of Contents
1.Load LLM (LlaMa-3)
2.Define Finance Tools
3.Create Finance Agent (LLM + Tools)

```

```


Finance Agent Functionalities

---


Stock Company Basic Information
Address, Industry, Sector, business summary, website, etc.
Top Company Officers
Stock Financials
Market Cap, current price, ebitda, total debt, total revenue, debt-to-equity, P/E ratio, operating margins, ebitda margins, operating cashflow, free cashflow, etc.
Stock Dividend & Earnings Dates
Information about Mutual Fund Holders
Information about Institutional Holders
Stock Splits History
Stock Ratings Upgrades & Downgrades by various Companies
News Articles about Stock

In [2]:
!pip install python-dotenv


In [3]:
import dotenv
import os

dotenv.load_dotenv(dotenv.find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]

KeyError: 'GROQ_API_KEY'

In [5]:

import yfinance as yf

yf.__version__

'0.2.50'

In [4]:
groq_api_key='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'

In [7]:
!pip install llama-index-llms-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [8]:
from llama_index.llms.groq import Groq

llama3 = Groq(model="llama-3.1-70b-versatile", api_key=groq_api_key, temperature=0) # llama3-70b-8192

llama3

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x79b9827c8f40>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x79b9765b67a0>, completion_to_prompt=<function default_completion_to_prompt at 0x79b97643f520>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama-3.1-70b-versatile', temperature=0.0, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [9]:
response = llama3.complete("Hey!! Whats up?")

print(response)

Not much. Just here and ready to chat. How about you? How's your day going so far?


#2. Define Tools

In [10]:
def flatten_dict(mapping: dict, prefix="")-> str:
    '''
    Flattens dictionary to string
    Example:
    Input -> {'name': 'Apple', 'Address': 'CA, US', 'closePrice': 229.9}
    Output ->
    name - Apple
    Address - CA, US
    closePrice - 229.9
    '''
    total_txt = ""
    for key, val in mapping.items():
        if type(val) in [str, float, int]:
            total_txt += f"{prefix}{key} - {val}\n"
        elif isinstance(val, dict):
            total_txt += f"{key}\n"
            total_txt += flatten_dict(val, prefix="\t")
        elif isinstance(val, list):
            if isinstance(val[0], dict):
                total_txt += f"{key}\n"
                for v in val:
                    total_txt += flatten_dict(v, prefix="\t")
            else:
                total_txt += f"{key} - {val}\n"
    return total_txt

In [11]:
from llama_index.core.tools import FunctionTool
from datetime import date

def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like addrress, industry, sector, company officers, business summary, website,
       marketCap, current Price, ebitda, total debt, total revenue, debt-to-equity, etc."""

    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return flatten_dict(ticker_info)

def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)

    mf_holders = ticker_obj.get_mutualfund_holders()
    return "\n\n".join([flatten_dict(record) for record in mf_holders.to_dict(orient="records")])

def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)

    inst_holders = ticker_obj.get_institutional_holders()
    return "\n\n".join([flatten_dict(record) for record in inst_holders.to_dict(orient="records")])

def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)

    curr_year = date.today().year

    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]

    return "\n\n".join([flatten_dict(record) for record in upgrades_downgrades.to_dict(orient="records")])

def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()

    return flatten_dict(hist_splits.to_dict())

def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    news_links = ticker_obj.get_news()

    for news in news_links:
        if "thumbnail" in news:
            news.pop("thumbnail")
        if "uuid" in news:
            news.pop("uuid")

    return "\n\n".join([flatten_dict(news) for news in news_links])


company_information_tool = FunctionTool.from_defaults(fn=company_information)
summary_of_mutual_fund_holders_tool = FunctionTool.from_defaults(fn=summary_of_mutual_fund_holders)
summary_of_institutional_holders_tool = FunctionTool.from_defaults(fn=summary_of_institutional_holders)
stock_grade_updrages_downgrades_tool = FunctionTool.from_defaults(fn=stock_grade_updrages_downgrades)
stock_splits_history_tool = FunctionTool.from_defaults(fn=stock_splits_history)
stock_news_tool = FunctionTool.from_defaults(fn=stock_news)

tools = [
    company_information_tool,
    summary_of_mutual_fund_holders_tool,
    summary_of_institutional_holders_tool,
    stock_grade_updrages_downgrades_tool,
    stock_splits_history_tool,
    stock_news_tool
]

In [12]:
print(company_information("AAPL"))

address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - (408) 996-1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensi

3. Define Finance Agent (LLM + Tools)[link text](https://raw.githubusercontent.com/sunny2309/llamaindex_tutorials/07988a4f31eed231043354a2da0b948c3fb93b3f/react.png)

In [13]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

finance_agent = ReActAgent.from_tools(tools=tools, llm=llama3, verbose=True)

finance_agent

In [14]:
response = finance_agent.query("What is address of microsoft?")

print(str(response))

> Running step 679e8c1a-0b1e-4b9f-a9e7-f710e30e6281. Step input: What is address of microsoft?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'MSFT'}
Observation: address1 - One Microsoft Way
city - Redmond
state - WA
zip - 98052-6399
country - United States
phone - 425 882 8080
website - https://www.microsoft.com
industry - Software - Infrastructure
industryKey - software-infrastructure
industryDisp - Software - Infrastructure
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office lice

In [15]:
response = finance_agent.query("What is last close price of microsoft?")

print(str(response))

> Running step 71b9a5a0-0395-419e-b70c-e6cae04991c0. Step input: What is last close price of microsoft?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'MSFT'}
Observation: address1 - One Microsoft Way
city - Redmond
state - WA
zip - 98052-6399
country - United States
phone - 425 882 8080
website - https://www.microsoft.com
industry - Software - Infrastructure
industryKey - software-infrastructure
industryDisp - Software - Infrastructure
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Of

In [16]:
response = finance_agent.query("What is dividend yield for apple stock?")

print(str(response))

> Running step acd414d5-0d10-4dc6-9940-196dc9f2609f. Step input: What is dividend yield for apple stock?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'AAPL'}
Observation: address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - (408) 996-1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provi

In [17]:
response = finance_agent.query("What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")

print(str(response))

> Running step 9e3a5152-6882-4e9c-a3df-b53c2d58ba90. Step input: What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'NVDA'}
Observation: address1 - 2788 San Tomas Expressway
city - Santa Clara
state - CA
zip - 95051
country - United States
phone - 408 486 2000
website - https://www.nvidia.com
industry - Semiconductors
industryKey - semiconductors
industryDisp - Semiconductors
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterpri

In [18]:
response = finance_agent.query("Summarize Key people at apple.")

print(str(response))

> Running step 5b567b60-268e-4ecf-9bcc-a99a7b5f1450. Step input: Summarize Key people at apple.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: company_information
Action Input: {'ticker': 'AAPL'}
Observation: address1 - One Apple Park Way
city - Cupertino
state - CA
zip - 95014
country - United States
phone - (408) 996-1010
website - https://www.apple.com
industry - Consumer Electronics
industryKey - consumer-electronics
industryDisp - Consumer Electronics
sector - Technology
sectorKey - technology
sectorDisp - Technology
longBusinessSummary - Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides Apple